In [ ]:
import requests

from bs4 import BeautifulSoup as bs
import os, sys
import re

from tqdm import tqdm
from datetime import datetime
import time

import pandas as pd

import pickle

# My functions
import lewisham_functions as lf

import functions as mf


In [ ]:
## Options
urlbase = "https://planning.lewisham.gov.uk/online-applications"

In [ ]:
# Postcode to search
postcode = ""

## Selenium option
Load up planning web page

In [ ]:
## Init browser object on planning page
browser = initbrowser(urlbase)

Enter in postcode details and search

In [ ]:
## With browser object make search
lf.makeSearch(postcode, browser)

### Submit search

In [ ]:
### SE263 only returned a single result.
# Check if only one application comes up
#def onSearchPage(browser):


# tag = browser.find_elements_by_xpath("//*[contains(text(), 'Application Summary')]")
# tag[0].text

#browser.find_elements_by_xpath("//*[contains(text(), 'Planning')]")
searchResults = lf.hasMultipleResults(browser)

In [ ]:
searchResults

Update search results to 100 on a page

In [ ]:

if searchResults:
    lf.makeResults100(browser)
#     numresults = browser.find_element_by_xpath("//select[@id='resultsPerPage']")
#     numresults.send_keys('100')
#     browser.find_element_by_xpath("//input[@type='submit']").click()


**Let's get the number of pages of results for this postcode and then the number of results in total**

In [ ]:

resultPages = lf.getResultNumber(browser, searchResults)


## Let's find the applications

### Extract applications via selenium

**First page**

Loop over each page

In [ ]:


# Iterate through all search pages and get links
hrefs = lf.getSearchResults(browser, searchResults, resultPages)

**Save pickled object**

In [ ]:

lf.saveLinks(hrefs, postcode) #os.path.abspath(os.curdir)

### Get application information via pickled object

In [ ]:
import imp
imp.reload(lf)
hrefs = lf.loadLinks(postcode)

## Let's get information on each application

In [ ]:
tabs = "Summary,Further Information,Contacts,Important Dates".split(',')

# Load up application
#app = hrefs[0]
#app
#browser.get(app)


In [ ]:
def GetTableFromPage(transp=True):
    
    """ Assumes a live browser object (selenium)
        returns a table extracted from the HTML
    """
    
    # Init table
    table = None
    
    # Let's load a table into a pd dataframe
    html = browser.page_source
    soup = bs(html, 'html.parser')
    div = soup.find('table')
    if div !=None:
        table = pd.read_html(str(div))[0]
        #table.iloc[:,0]
        #table.columns = table.iloc[:,0]
        #table.drop(index=0, inplace=True)
        if transp:
            table = table.transpose() 
        table.columns =table.iloc[0,:]
        table.drop(index=0, inplace=True)
    return table

In [ ]:
def getDetailsMultiplePages(links):
    
    """
        Loops over multiple pages to get information from the links
    """
    
    ## Initialise dataframe
    df = pd.DataFrame()
    
    for link in tqdm(links):

        # Load up application
        app = link
        
        try:
            browser.get(app)

            ## Initiliase table
            t1 = GetTableFromPage()

            ## Save URL
            t1.url = app

            ## Iterate over remaining tab names
            for t in tabs[1:]:

                #print(t)
                xp = "//span[contains(text(), '{}')]".format(t)

                btn = browser.find_element_by_xpath(xp)
                tabLink = btn.find_element_by_xpath('./..').get_attribute('href')
                browser.get(tabLink)
                newTable = GetTableFromPage()

                if str(type(newTable)) == "<class 'pandas.core.frame.DataFrame'>":
                    ## Update column names if already in table
                    newColumns = ['{}_{}'.format(t.replace(' ', '_'), c) if c in t1.columns else c for c in newTable.columns]
                    newTable.columns = newColumns

                    t1= t1.merge(newTable, 'outer', left_index=True, right_index=True)

            # Remove all spaces from col names
            newColumns = [col.replace(' ', '_') for col in t1.columns]
            t1.columns = newColumns

            ## If df hasn't been updated yet
            if df.shape[0] == 0:
                df = t1.copy()
            else:

                ## Add on row
                df = df.append(t1, sort=False)

            time.sleep(1)
        except: 
            pass
        
        
    return df
            
        



In [ ]:
import pdb
# Let's run loop over each applications
df =getDetailsMultiplePages(hrefs)
df.shape

In [ ]:
df.shape

**Let's save this object to pickle**

In [ ]:
def saveApplicationInfo(df):
    
    """ Save the application information that has been scraped by getDetailsMultiplePages """
    
    fname = "data_{}_{}.p".format(postcode, datetime.today().strftime('%Y%m%d'))
    
    if os.path.exists(fname):
        print("File '{}' already exists'")
    else:

        with open(fname, 'wb') as f: pickle.dump(df, f)


In [ ]:
saveApplicationInfo(df)

In [ ]:
def getPDFs(link):
    
    ####################################
    # Testing out get the documents etc.
    ####################################
    
    ## Get page for application (loads up 'Summary tab')
    browser.get(link)
    
    # Get case number
    caseTag = browser.find_element_by_xpath("//span[@class='caseNumber']").text
    
    
    # Table
    tabs = browser.find_element_by_xpath("//ul[@class='tabs']")
    
    lists_all = tabs.find_elements_by_xpath(".//li")

    ## Get direct children of section (where parents match)
    lists = [l for l in lists_all if l.find_element_by_xpath("./..") == tabs]

    #[l.text for l in lists]

    ##############################
    # Load documents page
    ##############################
    
    # How many documents?
    numDocs = int(re.findall('[0-9]+', lists[3].text)[0])
    if numDocs ==0:
        print("Jobby")
        return None
    else:
        lists[3].click()

        # Table with documents - no need to transpose before taking column names
        docTable = GetTableFromPage(transp=False)

        ## With PDF table assuming they are PDFs get the links
        htmlTable = browser.find_element_by_xpath("//table[@id='Documents']")

        ## Assuming all docs are pdfs
        links = htmlTable.find_elements_by_xpath(".//a[contains(@href, 'pdf')]")
        #len(links)

        ## Click each of the files to download
        buttons = browser.find_elements_by_xpath("//input[contains(@onclick, 'buttonSwitch')]")
        for b in buttons:

            b.click()

        # Download as .zip
        button = browser.find_element_by_xpath("//button[@type='submit'][@id='downloadFiles']")
        button.click()
        time.sleep(1)

        ## Get latest file 
        import glob 
        import shutil

        dloadfiles = glob.glob(os.path.join(r'C:\\', 'users', 'andre', 'Downloads', '*.zip'))


        latest_file = max(dloadfiles, key=os.path.getctime)
        fname = latest_file.split('\\')[-1]
        ## What is the full path?
        os.path.abspath(latest_file)
        ## Now move the file to current directory
        shutil.move(latest_file, os.path.join(os.curdir, '.downloads', '{}_{}'.format(postcode,fname)))


        # You've saved the PDFs, now keep the table
        return docTable

In [ ]:
import pdb
for l in tqdm(hrefs[6:]):
    getPDFs(l)
    time.sleep(2)

In [ ]:

## Iterate over tab names
for t in tabs:

    print(t)
    xp = "//span[contains(text(), '{}')]".format(t)
    #print(xp)
    btn = browser.find_element_by_xpath(xp)
    tabLink = btn.find_element_by_xpath('./..').get_attribute('href')
    browser.get(tabLink)
    newTable = GetTableFromPage()
    
    ## Update column names if already in table
    newColumns = ['{}_{}'.format(t.replace(' ', '_'), c) if c in t1.columns else c for c in newTable.columns]
    newTable.columns = newColumns
    #print(t1.columns)
    t1 = t1.merge(newTable, 'outer', left_index=True, right_index=True)
    print(t1.shape)


